In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

import pandas as pd
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

import os
from datetime import datetime

from ff_app.execution import run_data_pull, update_google_sheet
from ff_app import data_prep

10:36:31 INFO:NumExpr defaulting to 8 threads.


In [2]:
week = 14

if week == 0:
    short_df, combined_df = run_data_pull(week=1)
    # short_df, combined_df, all_df = run_data_pull(week=week, return_all_games=True)
    
    short_df = short_df[short_df['Datetime'] < '08/31']
    combined_df = combined_df[combined_df['Datetime'] < '08/31']
    short_df['Week'] = 0
    combined_df['Week'] = 0
    
    print(short_df.shape)
    print(combined_df.shape)

elif week == 1:
    short_df, combined_df = run_data_pull(week=1)
    # short_df, combined_df, all_df = run_data_pull(week=week, return_all_games=True)
    
    short_df = short_df[short_df['Datetime'] > '08/31']
    combined_df = combined_df[combined_df['Datetime'] > '08/31']
    print(short_df.shape)
    print(combined_df.shape)
    
else:
    short_df, combined_df = run_data_pull(week=week)
    # short_df, combined_df, all_df = run_data_pull(week=week, return_all_games=True)
    print(short_df.shape)
    print(combined_df.shape)

10:36:48 DEBUG:Starting new HTTP connection (1): www.espn.com:80
10:36:48 DEBUG:http://www.espn.com:80 "GET /college-football/scoreboard/_/group/80/year/2022/seasontype/2/week/14 HTTP/1.1" 301 241
10:36:48 DEBUG:Starting new HTTPS connection (1): www.espn.com:443
10:36:48 DEBUG:https://www.espn.com:443 "GET /college-football/scoreboard/_/group/80/year/2022/seasontype/2/week/14 HTTP/1.1" 200 None
10:36:48 INFO:Pulled 11 total games for week 14 in 2022
10:36:48 INFO:11 games have odds available
10:36:48 INFO:Picks sheet data has shape (11, 9)
10:36:48 INFO:Game list data has shape (11, 27)
10:36:48 INFO:Saving game data to disk at '/Users/hoke/dev/football/game_lists/2022/week14.csv'


(11, 9)
(11, 27)


In [3]:
short_df.head()

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
0,14,12/02 01:00 PM (Fri),,BUFF,vs,AKR,-11.5,55.5,BUFF 33 - AKR 22
1,14,12/02 07:30 PM (Fri),,UTSA,vs,UNT,-8.5,68.0,UTSA 38 - UNT 29
2,14,12/02 08:00 PM (Fri),Y,USC,vs,UTAH,-3.0,67.0,USC 35 - UTAH 32
3,14,12/03 12:00 PM (Sat),Y,TCU,vs,KSU,-2.5,62.0,TCU 32 - KSU 29
4,14,12/03 12:00 PM (Sat),,TOL,@,OHIO,-2.5,55.0,TOL 28 - OHIO 26


In [4]:
short_df[(short_df['Favorite'] == 'CLEM') | (short_df['Underdog'] == 'CLEM')]

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
10,14,12/03 08:00 PM (Sat),Y,CLEM,@,UNC,-7.5,63.5,CLEM 35 - UNC 28


In [5]:
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
2,14,12/02 08:00 PM (Fri),Y,USC,vs,UTAH,-3.0,67.0,USC 35 - UTAH 32
3,14,12/03 12:00 PM (Sat),Y,TCU,vs,KSU,-2.5,62.0,TCU 32 - KSU 29
6,14,12/03 04:00 PM (Sat),Y,UGA,vs,LSU,-17.5,51.0,UGA 34 - LSU 16
7,14,12/03 04:00 PM (Sat),Y,TULN,vs,UCF,-3.0,56.5,TULN 29 - UCF 26
10,14,12/03 08:00 PM (Sat),Y,CLEM,@,UNC,-7.5,63.5,CLEM 35 - UNC 28


In [6]:
team_list = ['MICH']

mandatory_picks = short_df[(short_df['Favorite'].isin(team_list)) |
                           (short_df['Underdog'].isin(team_list))]
mandatory_picks

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
9,14,12/03 08:00 PM (Sat),,MICH,vs,PUR,-16.5,51.5,MICH 34 - PUR 17


In [7]:
short_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
combined_df.loc[mandatory_picks.index.tolist(), 'Mandatory'] = 'Y'
short_df[short_df['Mandatory'] == 'Y']

,Week,Datetime,Mandatory,Favorite,Location,Underdog,Spread,Total,Implied Score
2,14,12/02 08:00 PM (Fri),Y,USC,vs,UTAH,-3.0,67.0,USC 35 - UTAH 32
3,14,12/03 12:00 PM (Sat),Y,TCU,vs,KSU,-2.5,62.0,TCU 32 - KSU 29
6,14,12/03 04:00 PM (Sat),Y,UGA,vs,LSU,-17.5,51.0,UGA 34 - LSU 16
7,14,12/03 04:00 PM (Sat),Y,TULN,vs,UCF,-3.0,56.5,TULN 29 - UCF 26
9,14,12/03 08:00 PM (Sat),Y,MICH,vs,PUR,-16.5,51.5,MICH 34 - PUR 17
10,14,12/03 08:00 PM (Sat),Y,CLEM,@,UNC,-7.5,63.5,CLEM 35 - UNC 28


In [8]:
update_google_sheet(short_df, combined_df)

11:00:56 INFO:Using credentials directory '/Users/hoke/dev/football/credentials'
11:00:56 INFO:Obtained credentails from existing 'token.json' file
11:00:56 INFO:Obtaining a token using credentials stored in 'credentials.json
11:00:56 INFO:Obtained a token using stored credentials
11:00:56 INFO:Writing data for Hoke
11:00:56 INFO:file_cache is only supported with oauth2client<4.0.0
11:00:56 DEBUG:Attempting to write 11 rows to Hoke!A:J
11:00:56 DEBUG:URL being requested: POST https://sheets.googleapis.com/v4/spreadsheets/1jlDdFE1zlYOVb535x15QVmEAlxB0rQYM6ARjY8vaQ3w/values/Hoke%21A%3AJ:append?valueInputOption=USER_ENTERED&alt=json
11:00:56 INFO:Attempting refresh to obtain initial access_token
11:00:56 DEBUG:[b'eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsImtpZCI6ImJkNDg3ZGZmOWMzMjZlNmIxMzY5MDllMzc5ODdiOTllMGJhODM0YjcifQ', b'eyJhdWQiOiJodHRwczovL29hdXRoMi5nb29nbGVhcGlzLmNvbS90b2tlbiIsInNjb3BlIjoiaHR0cHM6Ly93d3cuZ29vZ2xlYXBpcy5jb20vYXV0aC9zcHJlYWRzaGVldHMiLCJpYXQiOjE2Njk3Mzc2NTYsImV4cCI6MTY2OTc0M